In [1]:
from langgraph.graph import StateGraph , START , END
from langchain_ollama import ChatOllama
from typing import TypedDict , Annotated
import operator
from pydantic import BaseModel , Field

In [2]:
llm = ChatOllama(model = 'llama3')

In [3]:
class ValidateOutput(BaseModel):
    feedback : str = Field(description = 'Detailed feedback for the essay')
    score : int = Field(description = 'Score out of 10' , ge = 0 , le = 10)

In [4]:
structured_llm = llm.with_structured_output(ValidateOutput)

In [5]:
essay = """Artificial Intelligence (AI) has rapidly evolved from a theoretical concept into a transformative force shaping nearly every aspect of modern life. From voice assistants and recommendation systems to autonomous vehicles and medical diagnostics, AI is no longer a vision of the future—it is the present. As technology continues to advance, the future of AI promises unprecedented opportunities, along with significant challenges that society must carefully navigate.

One of the most impactful areas of AI’s future lies in automation and productivity. AI-powered systems will increasingly handle repetitive and time-consuming tasks across industries such as manufacturing, finance, logistics, and customer service. This will allow humans to focus on creative, strategic, and emotionally intelligent work. Rather than replacing humans entirely, AI is expected to work alongside them as an intelligent collaborator, enhancing efficiency and decision-making.

In the field of healthcare, AI has the potential to revolutionize diagnosis, treatment, and patient care. Advanced machine learning models can analyze medical images, predict disease outbreaks, and assist doctors in creating personalized treatment plans. In the future, AI-driven healthcare systems could make high-quality medical services more accessible and affordable, especially in remote and underserved regions.

Education will also undergo a major transformation. AI tutors and adaptive learning platforms will tailor education to individual students’ needs, learning speeds, and interests. This personalized approach can bridge learning gaps, improve engagement, and promote lifelong learning. Teachers will increasingly rely on AI for assessment and curriculum planning, allowing them to focus more on mentorship and creativity.

However, the future of AI also raises serious ethical and social concerns. Issues such as data privacy, algorithmic bias, job displacement, and the misuse of AI for surveillance or misinformation must be addressed. Transparent, fair, and accountable AI systems will be essential to build trust. Governments, researchers, and organizations must work together to create strong regulations and ethical frameworks that ensure AI benefits humanity as a whole.

Another critical aspect of AI’s future is the development of Artificial General Intelligence (AGI)—systems capable of performing any intellectual task a human can do. While AGI remains a long-term goal, its potential impact is enormous. If achieved responsibly, AGI could help solve global challenges such as climate change, poverty, and scientific discovery. However, uncontrolled or misaligned AI could pose serious risks, making safety research and alignment crucial.

In conclusion, the future of Artificial Intelligence is both exciting and complex. AI has the power to improve lives, drive innovation, and reshape economies, but its success depends on how responsibly it is developed and deployed. By prioritizing ethics, collaboration, and human-centered design, society can ensure that AI becomes a force for progress rather than disruption. The future of AI is not just about smarter machines—it is about creating a better, more inclusive world."""

In [22]:
structured_llm.invoke(f"Evaluate the following essay and calculate score out of 10 - {essay}")

ValidateOutput(feedback='very good', score=9)

In [ ]:
#state
class LLMState(TypedDict):
    
    essay_text : str
    cot_feedback : str # clarity of thought
    doa_feedback : str #depth of analysis
    language_feedback : str # check how is the language written by user
    final_summary : str
    ind_score : Annotated[list[int] , operator.add]  # concept of reducer to handle the individual score in parallel workflows
    final_score : float

In [8]:
def cot_feedback(state : LLMState) -> LLMState :

    prompt = f"Evaluate the clarity of thought of the following essay and generate a feedback and assign a score out of 10 \n {state['essay_text']}"

    output = structured_llm.invoke(prompt)

    return {'cot_feedback' : output.feedback , 'ind_score' : [output.score]}

In [9]:
def doa_feedback(state : LLMState) -> LLMState :

    prompt = f"Evaluate the depth of analysis of the following essay and generate a feedback and assign a score out of 10 \n {state['essay_text']}"

    output = structured_llm.invoke(prompt)

    return {'doa_feedback' : output.feedback , 'ind_score' : [output.score]}

In [10]:
def language_feedback(state : LLMState) -> LLMState :

    prompt = f"Evaluate the language quality of the following essay and generate a feedback and assign a score out of 10 \n {state['essay_text']}"

    output = structured_llm.invoke(prompt)

    return {'language_feedback' : output.feedback , 'ind_score' : [output.score]}


In [11]:
def final_summary(state : LLMState) -> LLMState :

    summary = f"Based on the following feedback , Give the Summarized feedback /n clarity of thought feedback - {state['cot_feedback']} /n depth of analysis feedback - {state['doa_feedback']} /n language feedback - {state['language_feedback']}"

    output = llm.invoke(summary)

    #calculate average score

    final_score = sum(state['ind_score'])/len(state['ind_score'])

    return {'final_summary' : output , 'final_score' : final_score}

In [12]:
#graph
graph = StateGraph(LLMState)

#nodes
graph.add_node('cot_feedback' , cot_feedback)
graph.add_node('doa_feedback' , doa_feedback)
graph.add_node('language_feedback' , language_feedback)
graph.add_node('final_summary' , final_summary)

#edges
graph.add_edge(START , 'cot_feedback')
graph.add_edge(START , 'doa_feedback')
graph.add_edge(START , 'language_feedback')
graph.add_edge('cot_feedback' , 'final_summary')
graph.add_edge('doa_feedback' , 'final_summary')
graph.add_edge('language_feedback' , 'final_summary')
graph.add_edge('final_summary' , END)

#compile
workflow = graph.compile()

In [13]:
essay = """Artificial Intelligence (AI) has rapidly evolved from a theoretical concept into a transformative force shaping nearly every aspect of modern life. From voice assistants and recommendation systems to autonomous vehicles and medical diagnostics, AI is no longer a vision of the future—it is the present. As technology continues to advance, the future of AI promises unprecedented opportunities, along with significant challenges that society must carefully navigate.

One of the most impactful areas of AI’s future lies in automation and productivity. AI-powered systems will increasingly handle repetitive and time-consuming tasks across industries such as manufacturing, finance, logistics, and customer service. This will allow humans to focus on creative, strategic, and emotionally intelligent work. Rather than replacing humans entirely, AI is expected to work alongside them as an intelligent collaborator, enhancing efficiency and decision-making.

In the field of healthcare, AI has the potential to revolutionize diagnosis, treatment, and patient care. Advanced machine learning models can analyze medical images, predict disease outbreaks, and assist doctors in creating personalized treatment plans. In the future, AI-driven healthcare systems could make high-quality medical services more accessible and affordable, especially in remote and underserved regions.

Education will also undergo a major transformation. AI tutors and adaptive learning platforms will tailor education to individual students’ needs, learning speeds, and interests. This personalized approach can bridge learning gaps, improve engagement, and promote lifelong learning. Teachers will increasingly rely on AI for assessment and curriculum planning, allowing them to focus more on mentorship and creativity.

However, the future of AI also raises serious ethical and social concerns. Issues such as data privacy, algorithmic bias, job displacement, and the misuse of AI for surveillance or misinformation must be addressed. Transparent, fair, and accountable AI systems will be essential to build trust. Governments, researchers, and organizations must work together to create strong regulations and ethical frameworks that ensure AI benefits humanity as a whole.

Another critical aspect of AI’s future is the development of Artificial General Intelligence (AGI)—systems capable of performing any intellectual task a human can do. While AGI remains a long-term goal, its potential impact is enormous. If achieved responsibly, AGI could help solve global challenges such as climate change, poverty, and scientific discovery. However, uncontrolled or misaligned AI could pose serious risks, making safety research and alignment crucial.

In conclusion, the future of Artificial Intelligence is both exciting and complex. AI has the power to improve lives, drive innovation, and reshape economies, but its success depends on how responsibly it is developed and deployed. By prioritizing ethics, collaboration, and human-centered design, society can ensure that AI becomes a force for progress rather than disruption. The future of AI is not just about smarter machines—it is about creating a better, more inclusive world."""

In [14]:
essay2 = """Artificial Intelligence is a very big technology and it is very future. AI is used everywhere like phones, computers, laptops, fridges, and also maybe in roads. The future of AI is very futureful and it will do many things that humans cannot do or maybe can do but AI does faster.

AI will be very important in the future because it is smart. Many companies use AI because it saves time and money and also it works 24 hours and never sleeps like humans who sleep a lot. In the future, AI will do all jobs and humans will relax or maybe become lazy, which is good and bad both.

In healthcare, AI will help doctors because doctors are humans and humans make mistakes. AI does not make mistakes (sometimes it does but mostly no). AI can check reports, scan images, and tell the disease very fast without even thinking. This is very helpful and very amazing.

Education will also change because AI can teach students better than teachers because teachers get angry sometimes but AI does not get angry unless it is programmed angry. Students will learn from AI and pass exams easily without studying much which is good for students.

There are some problems also like privacy and jobs but these problems are not that big because technology always comes and people adjust. Even when computers came people were scared but now everyone uses computers so AI is same thing.

In conclusion, the future of AI is very future and very big. AI will grow more and more and more and humans will use it more and more. AI is the future and the future is AI and that is why AI future is future."""

In [15]:
#execute
initial_state = {'essay_text' : essay}

final_state = workflow.invoke(initial_state)

In [16]:
#output
print(final_state)
print(final_state['ind_score'])
print(final_state['final_score'])
print(final_state['final_summary'])

{'essay_text': 'Artificial Intelligence (AI) has rapidly evolved from a theoretical concept into a transformative force shaping nearly every aspect of modern life. From voice assistants and recommendation systems to autonomous vehicles and medical diagnostics, AI is no longer a vision of the future—it is the present. As technology continues to advance, the future of AI promises unprecedented opportunities, along with significant challenges that society must carefully navigate.\n\nOne of the most impactful areas of AI’s future lies in automation and productivity. AI-powered systems will increasingly handle repetitive and time-consuming tasks across industries such as manufacturing, finance, logistics, and customer service. This will allow humans to focus on creative, strategic, and emotionally intelligent work. Rather than replacing humans entirely, AI is expected to work alongside them as an intelligent collaborator, enhancing efficiency and decision-making.\n\nIn the field of healthca

In [17]:
#execute
initial_state = {'essay_text' : essay2}

final_state = workflow.invoke(initial_state)

In [18]:
#output
print(final_state)
print(final_state['ind_score'])
print(final_state['final_score'])
print(final_state['final_summary'])

{'essay_text': 'Artificial Intelligence is a very big technology and it is very future. AI is used everywhere like phones, computers, laptops, fridges, and also maybe in roads. The future of AI is very futureful and it will do many things that humans cannot do or maybe can do but AI does faster.\n\nAI will be very important in the future because it is smart. Many companies use AI because it saves time and money and also it works 24 hours and never sleeps like humans who sleep a lot. In the future, AI will do all jobs and humans will relax or maybe become lazy, which is good and bad both.\n\nIn healthcare, AI will help doctors because doctors are humans and humans make mistakes. AI does not make mistakes (sometimes it does but mostly no). AI can check reports, scan images, and tell the disease very fast without even thinking. This is very helpful and very amazing.\n\nEducation will also change because AI can teach students better than teachers because teachers get angry sometimes but AI

In [19]:
essay3 = """Artificial Intelligence is one of the most important technology of modern world and it is changing the way humans are working and living. AI already use in many areas like healthcare, education, industries and daily life applications. In coming future, AI will become more powerful and more helpful for solving complex human problems.

One major impact of AI will be in automation and productivity where machines will doing repetitive task faster and more accurate than humans. This will helping companies to save time and cost while humans can focus on creative and decision making works. AI not replacing humans fully but it will work together with humans for better output.

Healthcare sector will also get huge benefit from Artificial Intelligence. AI system can analyzing medical images, predicting disease earlier and helping doctors in treatment decision. This will reducing human error and improving patient care quality. In future AI based healthcare will become more accessible even in rural areas.

Education will also changing due to AI technology. Personalized learning system powered by AI will help students learn according to their speed and interest. Teachers can use AI tools for evaluation and lesson planning which will saving time. This will making education more effective and engaging for students.

However, there are some challenges also related to AI future. Issues like data privacy, job displacement and ethical misuse of AI need to be addressed properly. Governments and organizations must creating rules and guidelines so that AI is used in responsible manner.

In conclusion, the future of Artificial Intelligence is very promising and powerful. If AI developed and used carefully, it can improving human life and solving global problems. The future of AI not only about machines but about how humans and technology grow together."""

In [20]:
#execute
initial_state = {'essay_text' : essay3}

final_state = workflow.invoke(initial_state)

In [21]:
#output
print(final_state)
print(final_state['ind_score'])
print(final_state['final_score'])
print(final_state['final_summary'])

{'essay_text': 'Artificial Intelligence is one of the most important technology of modern world and it is changing the way humans are working and living. AI already use in many areas like healthcare, education, industries and daily life applications. In coming future, AI will become more powerful and more helpful for solving complex human problems.\n\nOne major impact of AI will be in automation and productivity where machines will doing repetitive task faster and more accurate than humans. This will helping companies to save time and cost while humans can focus on creative and decision making works. AI not replacing humans fully but it will work together with humans for better output.\n\nHealthcare sector will also get huge benefit from Artificial Intelligence. AI system can analyzing medical images, predicting disease earlier and helping doctors in treatment decision. This will reducing human error and improving patient care quality. In future AI based healthcare will become more acc